In [ ]:
!pip install gradio

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from io import open
pd.options.display.max_columns = 150

In [ ]:
data = pd.read_csv('data-final.csv', sep='\t')

In [ ]:
data.head()

In [ ]:
data.drop(data.columns[50:110], axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
pd.options.display.float_format = "{:.2f}".format
data.describe()

In [ ]:
data["EXT1"].value_counts()

In [ ]:
data[(data == 0.00).all(axis=1)].describe()

In [ ]:
data = data[(data > 0.00).all(axis=1)]

In [ ]:
data["EXT1"].value_counts()

In [ ]:
!pip install yellowbrick

In [ ]:
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

In [ ]:
kmeans = KMeans()
visualizer = KElbowVisualizer(kmeans, k=(2,10))

In [ ]:
data_sample = data.sample(n=5000, random_state=1)

In [ ]:
visualizer.fit(data_sample)
visualizer.poof()

In [ ]:
kmeans = KMeans(n_clusters=5)
k_fit = kmeans.fit(data)

In [ ]:
predicoes = k_fit.labels_
data['Clusters'] =predicoes

In [ ]:
data.head()

In [ ]:
data["Clusters"].value_counts()

In [ ]:
data.groupby('Clusters').mean()

In [ ]:
col_list = list(data)
ext = col_list[0:10]
est = col_list[10:20]
agr = col_list[20:30]
csn = col_list[30:40]
opn = col_list[40:50]

In [ ]:
data_soma = pd.DataFrame()
data_soma['extroversion'] = data[ext].sum(axis=1)/10
data_soma['neurotic'] = data[est].sum(axis=1)/10
data_soma['agreeable'] = data[agr].sum(axis=1)/10
data_soma['conscientious'] = data[csn].sum(axis=1)/10
data_soma['open'] = data[opn].sum(axis=1)/10
data_soma['Clusters'] = predicoes 


In [ ]:
data_soma.groupby('Clusters').mean()

In [ ]:
data_clusters = data_soma.groupby('Clusters').mean()

In [ ]:
plt.figure(figsize=(22,3))
for i in range(0, 5):
    plt.subplot(1,5,i+1)
    plt.bar(data_clusters.columns, data_clusters.iloc[:, i], color='green', alpha=0.2)
    plt.plot(data_clusters.columns, data_clusters.iloc[:, i], color='red')
    plt.title('Grupo' + str(i))
    plt.xticks(rotation=45)
    plt.ylim(0,4)

In [ ]:
data[:0].to_excel("perguntas.xlsx",index=False)

In [ ]:
meus_dados = pd.read_excel('perguntas.xlsx')

In [ ]:
grupo_personalidade = k_fit.predict(meus_dados)[0] 
print('Meu grupo de pernonalidade é:', grupo_personalidade)

In [ ]:
import gradio as gr

In [ ]:
dicio_questions = open("questions.txt").read().split("\n")

In [ ]:
dicio_questions

In [ ]:
questions = []
for q in dicio_questions:
    q = str(q)
    questions.append(q[q.find("\t"):].lstrip())

In [ ]:
questions

In [ ]:
inputs_questions = []
for q in questions:
    obj_input = gr.inputs.Slider(minimum=1,maximum=5,step=1,default=3,label=q)
    inputs_questions.append(obj_input)

In [ ]:
 
def predict(*outputs_questions):
    outputs_questions = np.array(outputs_questions).reshape(1, -1)
    return k_fit.predict(outputs_questions)

iface = gr.Interface(
    fn = predict,
    title = "Big Five Personality",
    description = "sistema para detecção de traços de personalidade",
    inputs = inputs_questions,
    outputs = "text")
iface.launch(share=True)